In [67]:
# For Reference 
# Date;Time;BTC/USD Bid;BTC/USD Bid Size;BTC/USD Ask;BTC/USD Ask Size;BTC/EUR Bid;BTC/EUR Bid Size;BTC/EUR Ask;BTC/EUR Ask Size;ETH/BTC Bid;ETH/BTC Bid Size;ETH/BTC Ask;ETH/BTC Ask Size;EUR/USD Bid;EUR/USD Bid Size;EUR/USD Ask;EUR/USD Ask Size;ETH/USD Bid;ETH/USD Bid Size;ETH/USD Ask;ETH/USD Ask Size;ETH/EUR Bid;ETH/EUR Bid Size;ETH/EUR Ask;ETH/EUR Ask Size

In [68]:
# Note datetime is the module name, but datetime is a class two. To call the datetime class use datetime.datetime
import datetime

In [69]:
def remove_from_str(s, substrings_to_remove) -> str:
    """
    Pretty inefficient but whatever.
    Use regex for speedup.
    """
    ret = s
    for sub in substrings_to_remove:
        ret = ret.replace(sub, "")
    return(ret)

In [70]:
def next_interval(t: datetime.datetime) -> datetime.datetime:
    """
    Return the next 5 minute interval
    """
    return(t + datetime.timedelta(minutes=5))

# print(datetime.datetime(2021, 9, 13, 12, 10))
# print(next_interval(datetime.datetime(2021, 9, 13, 12, 10)))

In [71]:
chars_to_remove = {'$', '€', ',', "BTC"}
f = open("Assignment 1 - Data.txt", 'r')
text = f.read()

rows = text.split('\n')

In [72]:
d = dict()
t = []
indexes = dict()

columns = rows[0].split(';')

for trade_type in columns[2:]:
    d[trade_type] = dict()

for i in range(len(columns)):
    indexes[i] = columns[i] 


In [73]:
#Setting up dictionary for easy lookup
for row in rows[1:]:
    split = row.split(';')
    for i in range(len(split)):
        if i == 0:
            date = split[i]
        elif i == 1:
            time_of_day = split[i]
        else:
            trade_time = datetime.datetime.strptime(date + ' ' + time_of_day, "%m/%d/%Y %I:%M %p")
            trade_type = indexes[i]
            d[trade_type][trade_time] = float(remove_from_str(split[i], chars_to_remove))

            

In [74]:
#Setting up times 
times = []
for i in d:
    for j in d[i]:
        times.append(j)
    break
times.sort()

In [75]:
# orders = ["BTC/USD Bid", "BTC/USD Ask", "BTC/EUR Bid", "BTC/EUR Ask", "ETH/BTC Bid", "ETH/BTC Ask", "EUR/USD Bid", "EUR/USD Ask", "ETH/USD Bid", "ETH/USD Ask", "ETH/EUR Bid", "ETH/EUR Ask"]

# Bid 
# We get the denomiator
# Ex BTC/USD Bid * BTC/USD Bid Size = $48,064.80 * 9.4296 in BTC to be sold 

# Ask
# We get the numerator
# BTC/USD Ask * BTC/USD Ask Size = $48,269.41 * 18.2923 in BTC to be bought
# trading 3 and four currencies

trade_set = [
["EUR/USD Ask", "BTC/EUR Ask", "ETH/BTC Ask", "ETH/USD Bid"], # USD -> EUR -> BTC -> ETH -> USD 
["EUR/USD Ask", "ETH/EUR Ask", "ETH/BTC Bid", "BTC/USD Bid"], # USD -> EUR -> ETH -> BTC -> USD 
["BTC/USD Ask", "BTC/EUR Bid", "ETH/EUR Ask", "ETH/USD Bid"], # USD -> BTC -> EUR -> ETH -> USD 
["BTC/USD Ask", "ETH/BTC Ask", "ETH/EUR Bid", "EUR/USD Bid"], # USD -> BTC -> ETH -> EUR -> USD 
["ETH/USD Ask", "ETH/BTC Bid", "BTC/EUR Bid", "EUR/USD Bid"], # USD -> ETH -> BTC -> EUR -> USD 
["ETH/USD Ask", "ETH/EUR Bid", "BTC/EUR Ask", "BTC/USD Bid"], # USD -> ETH -> EUR -> BTC -> USD 
["EUR/USD Ask", "BTC/EUR Ask", "BTC/USD Bid"], # USD -> EUR -> BTC -> USD
["EUR/USD Ask", "ETH/EUR Ask", "ETH/USD Bid"], # USD -> EUR -> ETH -> USD
["BTC/USD Ask", "BTC/EUR Bid", "EUR/USD Bid"], # USD -> BTC -> EUR -> USD
["BTC/USD Ask", "ETH/BTC Ask", "ETH/USD Bid"], # USD -> BTC -> ETH -> USD
["ETH/USD Ask", "ETH/EUR Bid", "EUR/USD Bid"], # USD -> ETH -> EUR -> USD
["ETH/USD Ask", "ETH/BTC Bid", "BTC/USD Bid"], # USD -> ETH -> BTC -> USD
]

portfolio_values = []

def new_method():
    portfolio_value = 1
    best_trades = None
    outstr = ""
    for t in times:
        max_profit = 0
        for trades in trade_set:
            end_value = portfolio_value
            shares = []
            for trade in trades:
                if "Bid" in trade:
                    end_value *= d[trade][t]
                    shares.append(end_value)
                elif "Ask" in trade: 
                    end_value /= d[trade][t]
                    shares.append(end_value)
            profit = end_value - portfolio_value
            sizes = []
            for trade in trades:
                sizes.append(d[trade + " Size"][t])
            ratios = [sizes[i] / shares[i] for i in range(0, len(shares))]
            minflow = min(ratios) if min(ratios) < 1 else 1
            profit = minflow * profit
            sizes = [min(shares[i], sizes[i]) for i in range(0, len(shares))]
            if profit > max_profit:
                max_profit = profit
                best_sizes = sizes
                best_trades = trades
        if best_trades is None:
            outstr += str(t.strftime("%m/%d/%y;%I:%M%p")) + ";BUY,;SELL,;$0;0" + '\n'
        else:
            buys = []
            sells = []
            for i in range(len(best_trades)):
                if "Bid" in best_trades[i]:
                    # This is a sell
                    sells.append((best_trades[i], best_sizes[i])) 
                elif "Ask" in best_trades[i]:
                    # This is a buy 
                    buys.append((best_trades[i], best_sizes[i]))

            buy_str = ";BUY,"
            for trade in buys:
                buy_str += "{}-{}".format(trade[0], trade[1])
            sell_str = ";SELL,"
            for trade in sells:
                sell_str += "{}-{}".format(trade[0], trade[1])
            outstr += str(t.strftime("%m/%d/%y;%I:%M%p")) + buy_str + sell_str + ";{};{}".format(max_profit, 0) + "\n"

        portfolio_value += max_profit
        portfolio_values.append(portfolio_value)

    return((portfolio_value, outstr, portfolio_values))
value, out, y = new_method()
print("${}".format(round(value, 2)))
f = open("output.txt", 'w')
f.write(out)
f.close()

$66590.09


In [76]:
import matplotlib.pyplot as plt
x = times

plt.plot(x, y)
plt.xlabel("Time (Month-Day Hour)")
plt.ylabel("Profit (USD)")
plt.title("Profit of Portfolio Over Time")
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig("fig.png", dpi = 500)
plt.show()


    